# Fornite Ranking 
Dataset created with API and WebScrapping

In [1]:
import os
import requests
import pandas as pd
import numpy as np
from dotenv import load_dotenv, find_dotenv
from bs4 import BeautifulSoup

## Web Scraping to get the top 100 players - Fortnite

In [2]:
#Web Scraping to get the top 100 Fortnite players (with most money)
response= requests.get('https://www.esportsearnings.com/games/534-fortnite/top-players')
html = response.content
soup = BeautifulSoup(html, "lxml")
table = pd.read_html(html, attrs={'class' : 'detail_list_table'})[0]
table

,Unnamed: 0,Player ID,Player Name,Total (Game),Total (Overall),% of Total
0,1.0,Bugha,Kyle Giersdorf,"$3,072,333.34","$3,072,333.34",100.00%
1,2.0,Aqua,David Wang,"$1,900,541.23","$1,900,541.23",100.00%
2,3.0,psalm,Harrison Chang,"$1,870,738.80","$1,954,950.00",95.69%
3,4.0,Nyhrox,Emil Bergquist Pedersen,"$1,532,845.69","$1,532,845.69",100.00%
4,5.0,EpikWhale,Shane Cotton,"$1,330,841.54","$1,330,841.54",100.00%
...,...,...,...,...,...,...
95,96.0,Tuckz,Jordan Tucker,"$131,750.00","$131,750.00",100.00%
96,97.0,Nicks,Nicollas Polonio de Oliveira,"$131,200.00","$131,200.00",100.00%
97,98.0,NICKMERCS,Nick Kolcheff,"$130,725.00","$167,125.01",78.22%
98,99.0,Gotaga,Corentin Houssein,"$129,758.22","$160,399.66",80.90%


In [3]:
#Create another column called "Name Correct" to enable the correct merge
table['Name Correct'] =[str.capitalize(i) for i in table['Player ID']]

In [4]:
table

,Unnamed: 0,Player ID,Player Name,Total (Game),Total (Overall),% of Total,Name Correct
0,1.0,Bugha,Kyle Giersdorf,"$3,072,333.34","$3,072,333.34",100.00%,Bugha
1,2.0,Aqua,David Wang,"$1,900,541.23","$1,900,541.23",100.00%,Aqua
2,3.0,psalm,Harrison Chang,"$1,870,738.80","$1,954,950.00",95.69%,Psalm
3,4.0,Nyhrox,Emil Bergquist Pedersen,"$1,532,845.69","$1,532,845.69",100.00%,Nyhrox
4,5.0,EpikWhale,Shane Cotton,"$1,330,841.54","$1,330,841.54",100.00%,Epikwhale
...,...,...,...,...,...,...,...
95,96.0,Tuckz,Jordan Tucker,"$131,750.00","$131,750.00",100.00%,Tuckz
96,97.0,Nicks,Nicollas Polonio de Oliveira,"$131,200.00","$131,200.00",100.00%,Nicks
97,98.0,NICKMERCS,Nick Kolcheff,"$130,725.00","$167,125.01",78.22%,Nickmercs
98,99.0,Gotaga,Corentin Houssein,"$129,758.22","$160,399.66",80.90%,Gotaga


## API to get the information about top players
Source: https://dash.fortnite-api.com/

In [5]:
#Applying the API for each Player ID
url = [f'https://fortnite-api.com/v1/stats/br/v2?name={name}' for name in table['Player ID']]
response = [requests.get(i).json() for i in url]

In [6]:
#Turn into a dataframe
x = pd.json_normalize(response)
pd.set_option('display.max_columns', None)

In [7]:
#Merge the scrap with API content
all_info = pd.merge(left=table, right=x, left_on='Name Correct', right_on='data.account.name')

In [8]:
#Data Cleaning
all_info.drop(['status','error', 'Name Correct','data.image'], inplace = True, axis = 1)
all_info.rename(columns = {'Unnamed: 0':'Rank Position'}, inplace = True)
columns_to_drop = list(all_info.loc[: ,all_info.isnull().mean()>0.9].columns)
all_info.drop(columns = columns_to_drop, inplace = True)

In [9]:
#Renaming Columns
rename_columns = [(' '.join(i.split(sep='.'))).replace('Data stats all','') for i in all_info.columns]
all_info.columns= rename_columns

In [10]:
all_info

,Rank Position,Player ID,Player Name,Total (Game),Total (Overall),% of Total,data account id,data account name,data battlePass level,data battlePass progress,data stats all overall score,data stats all overall scorePerMin,data stats all overall scorePerMatch,data stats all overall wins,data stats all overall top3,data stats all overall top5,data stats all overall top6,data stats all overall top10,data stats all overall top12,data stats all overall top25,data stats all overall kills,data stats all overall killsPerMin,data stats all overall killsPerMatch,data stats all overall deaths,data stats all overall kd,data stats all overall matches,data stats all overall winRate,data stats all overall minutesPlayed,data stats all overall playersOutlived,data stats all overall lastModified,data stats all solo score,data stats all solo scorePerMin,data stats all solo scorePerMatch,data stats all solo wins,data stats all solo top10,data stats all solo top25,data stats all solo kills,data stats all solo killsPerMin,data stats all solo killsPerMatch,data stats all solo deaths,data stats all solo kd,data stats all solo matches,data stats all solo winRate,data stats all solo minutesPlayed,data stats all solo playersOutlived,data stats all solo lastModified,data stats all duo score,data stats all duo scorePerMin,data stats all duo scorePerMatch,data stats all duo wins,data stats all duo top5,data stats all duo top12,data stats all duo kills,data stats all duo killsPerMin,data stats all duo killsPerMatch,data stats all duo deaths,data stats all duo kd,data stats all duo matches,data stats all duo winRate,data stats all duo minutesPlayed,data stats all duo playersOutlived,data stats all duo lastModified,data stats all trio score,data stats all trio scorePerMin,data stats all trio scorePerMatch,data stats all trio wins,data stats all trio top3,data stats all trio top6,data stats all trio kills,data stats all trio killsPerMin,data stats all trio killsPerMatch,data stats all trio deaths,data stats all trio kd,data stats all trio matches,data stats all trio winRate,data stats all trio minutesPlayed,data stats all trio playersOutlived,data stats all trio lastModified,data stats all squad score,data stats all squad scorePerMin,data stats all squad scorePerMatch,data stats all squad wins,data stats all squad top3,data stats all squad top6,data stats all squad kills,data stats all squad killsPerMin,data stats all squad killsPerMatch,data stats all squad deaths,data stats all squad kd,data stats all squad matches,data stats all squad winRate,data stats all squad minutesPlayed,data stats all squad playersOutlived,data stats all squad lastModified,data stats all ltm score,data stats all ltm scorePerMin,data stats all ltm scorePerMatch,data stats all ltm wins,data stats all ltm kills,data stats all ltm killsPerMin,data stats all ltm killsPerMatch,data stats all ltm deaths,data stats all ltm kd,data stats all ltm matches,data stats all ltm winRate,data stats all ltm minutesPlayed,data stats all ltm playersOutlived,data stats all ltm lastModified,data stats keyboardMouse overall score,data stats keyboardMouse overall scorePerMin,data stats keyboardMouse overall scorePerMatch,data stats keyboardMouse overall wins,data stats keyboardMouse overall top3,data stats keyboardMouse overall top5,data stats keyboardMouse overall top6,data stats keyboardMouse overall top10,data stats keyboardMouse overall top12,data stats keyboardMouse overall top25,data stats keyboardMouse overall kills,data stats keyboardMouse overall killsPerMin,data stats keyboardMouse overall killsPerMatch,data stats keyboardMouse overall deaths,data stats keyboardMouse overall kd,data stats keyboardMouse overall matches,data stats keyboardMouse overall winRate,data stats keyboardMouse overall minutesPlayed,data stats keyboardMouse overall playersOutlived,data stats keyboardMouse overall lastModified,data stats keyboardMouse solo score,data stats keyboardMouse solo scorePerMin,data stats keyboardMous

In [11]:
#Top players - Export to CSV
all_info.to_csv('./Datasets/top_players', index=False)

# Using the API to get friends information

In [12]:
friends=['Floelr', 'Monsterbale', 'Sharshock','BEP Rodolfo477','williamtotake','BzAlva','Cauanyca','Guibru123']
results = [requests.get(f'https://fortnite-api.com/v1/stats/br/v2?name={i}').json() for i in friends]

In [13]:
friends_compare= pd.json_normalize(results)

In [14]:
#Data Cleaning
columns_to_drop = list(friends_compare.loc[: ,friends_compare.isnull().mean()>0.9].columns)
friends_compare.drop(columns=columns_to_drop, inplace=True)

In [15]:
#Renaming Columns
rename_columns = [(' '.join(i.split(sep='.'))).replace('data stats all','') for i in friends_compare.columns]
friends_compare.columns= rename_columns

In [16]:
friends_compare

,status,data account id,data account name,data battlePass level,data battlePass progress,overall score,overall scorePerMin,overall scorePerMatch,overall wins,overall top3,overall top5,overall top6,overall top10,overall top12,overall top25,overall kills,overall killsPerMin,overall killsPerMatch,overall deaths,overall kd,overall matches,overall winRate,overall minutesPlayed,overall playersOutlived,overall lastModified,solo score,solo scorePerMin,solo scorePerMatch,solo wins,solo top10,solo top25,solo kills,solo killsPerMin,solo killsPerMatch,solo deaths,solo kd,solo matches,solo winRate,solo minutesPlayed,solo playersOutlived,solo lastModified,duo score,duo scorePerMin,duo scorePerMatch,duo wins,duo top5,duo top12,duo kills,duo killsPerMin,duo killsPerMatch,duo deaths,duo kd,duo matches,duo winRate,duo minutesPlayed,duo playersOutlived,duo lastModified,trio score,trio scorePerMin,trio scorePerMatch,trio wins,trio top3,trio top6,trio kills,trio killsPerMin,trio killsPerMatch,trio deaths,trio kd,trio matches,trio winRate,trio minutesPlayed,trio playersOutlived,trio lastModified,squad score,squad scorePerMin,squad scorePerMatch,squad wins,squad top3,squad top6,squad kills,squad killsPerMin,squad killsPerMatch,squad deaths,squad kd,squad matches,squad winRate,squad minutesPlayed,squad playersOutlived,squad lastModified,ltm score,ltm scorePerMin,ltm scorePerMatch,ltm wins,ltm kills,ltm killsPerMin,ltm killsPerMatch,ltm deaths,ltm kd,ltm matches,ltm winRate,ltm minutesPlayed,ltm playersOutlived,ltm lastModified,data stats gamepad overall score,data stats gamepad overall scorePerMin,data stats gamepad overall scorePerMatch,data stats gamepad overall wins,data stats gamepad overall top3,data stats gamepad overall top5,data stats gamepad overall top6,data stats gamepad overall top10,data stats gamepad overall top12,data stats gamepad overall top25,data stats gamepad overall kills,data stats gamepad overall killsPerMin,data stats gamepad overall killsPerMatch,data stats gamepad overall deaths,data stats gamepad overall kd,data stats gamepad overall matches,data stats gamepad overall winRate,data stats gamepad overall minutesPlayed,data stats gamepad overall playersOutlived,data stats gamepad overall lastModified,data stats gamepad solo score,data stats gamepad solo scorePerMin,data stats gamepad solo scorePerMatch,data stats gamepad solo wins,data stats gamepad solo top10,data stats gamepad solo top25,data stats gamepad solo kills,data stats gamepad solo killsPerMin,data stats gamepad solo killsPerMatch,data stats gamepad solo deaths,data stats gamepad solo kd,data stats gamepad solo matches,data stats gamepad solo winRate,data stats gamepad solo minutesPlayed,data stats gamepad solo playersOutlived,data stats gamepad solo lastModified,data stats gamepad duo score,data stats gamepad duo scorePerMin,data stats gamepad duo scorePerMatch,data stats gamepad duo wins,data stats gamepad duo top5,data stats gamepad duo top12,data stats gamepad duo kills,data stats gamepad duo killsPerMin,data stats gamepad duo killsPerMatch,data stats gamepad duo deaths,data stats gamepad duo kd,data stats gamepad duo matches,data stats gamepad duo winRate,data stats gamepad duo minutesPlayed,data stats gamepad duo playersOutlived,data stats gamepad duo lastModified,data stats gamepad trio score,data stats gamepad trio scorePerMin,data stats gamepad trio scorePerMatch,data stats gamepad trio wins,data stats gamepad trio top3,data stats gamepad trio top6,data stats gamepad trio kills,data stats gamepad trio killsPerMin,data stats gamepad trio killsPerMatch,data stats gamepad trio deaths,data stats gamepad trio kd,data stats gamepad trio matches,data stats gamepad trio winRate,data stats gamepad trio minutesPlayed,data stats gamepad trio playersOutlived,data stats gamepad trio lastModified,data stats gamepad squad score,data stats gamepad squad scorePerMin,data stats gamepad squad scorePerMatch,data stats gamepad squad wins,data stats gamepad squad top3,data stats

In [17]:
#Top friends - Export to CSV
table.to_csv('./Datasets/top_friends', index=False)

In [18]:
gather = pd.concat([all_info, friends_compare]).reset_index()
gather.reset_index(drop=True)
#gather.drop(columns=['Rank Position','Player ID','Player Name', 'Total (Game)','Total (Overall)','% of Total'], inplace=True)
gather

,index,Rank Position,Player ID,Player Name,Total (Game),Total (Overall),% of Total,data account id,data account name,data battlePass level,data battlePass progress,data stats all overall score,data stats all overall scorePerMin,data stats all overall scorePerMatch,data stats all overall wins,data stats all overall top3,data stats all overall top5,data stats all overall top6,data stats all overall top10,data stats all overall top12,data stats all overall top25,data stats all overall kills,data stats all overall killsPerMin,data stats all overall killsPerMatch,data stats all overall deaths,data stats all overall kd,data stats all overall matches,data stats all overall winRate,data stats all overall minutesPlayed,data stats all overall playersOutlived,data stats all overall lastModified,data stats all solo score,data stats all solo scorePerMin,data stats all solo scorePerMatch,data stats all solo wins,data stats all solo top10,data stats all solo top25,data stats all solo kills,data stats all solo killsPerMin,data stats all solo killsPerMatch,data stats all solo deaths,data stats all solo kd,data stats all solo matches,data stats all solo winRate,data stats all solo minutesPlayed,data stats all solo playersOutlived,data stats all solo lastModified,data stats all duo score,data stats all duo scorePerMin,data stats all duo scorePerMatch,data stats all duo wins,data stats all duo top5,data stats all duo top12,data stats all duo kills,data stats all duo killsPerMin,data stats all duo killsPerMatch,data stats all duo deaths,data stats all duo kd,data stats all duo matches,data stats all duo winRate,data stats all duo minutesPlayed,data stats all duo playersOutlived,data stats all duo lastModified,data stats all trio score,data stats all trio scorePerMin,data stats all trio scorePerMatch,data stats all trio wins,data stats all trio top3,data stats all trio top6,data stats all trio kills,data stats all trio killsPerMin,data stats all trio killsPerMatch,data stats all trio deaths,data stats all trio kd,data stats all trio matches,data stats all trio winRate,data stats all trio minutesPlayed,data stats all trio playersOutlived,data stats all trio lastModified,data stats all squad score,data stats all squad scorePerMin,data stats all squad scorePerMatch,data stats all squad wins,data stats all squad top3,data stats all squad top6,data stats all squad kills,data stats all squad killsPerMin,data stats all squad killsPerMatch,data stats all squad deaths,data stats all squad kd,data stats all squad matches,data stats all squad winRate,data stats all squad minutesPlayed,data stats all squad playersOutlived,data stats all squad lastModified,data stats all ltm score,data stats all ltm scorePerMin,data stats all ltm scorePerMatch,data stats all ltm wins,data stats all ltm kills,data stats all ltm killsPerMin,data stats all ltm killsPerMatch,data stats all ltm deaths,data stats all ltm kd,data stats all ltm matches,data stats all ltm winRate,data stats all ltm minutesPlayed,data stats all ltm playersOutlived,data stats all ltm lastModified,data stats keyboardMouse overall score,data stats keyboardMouse overall scorePerMin,data stats keyboardMouse overall scorePerMatch,data stats keyboardMouse overall wins,data stats keyboardMouse overall top3,data stats keyboardMouse overall top5,data stats keyboardMouse overall top6,data stats keyboardMouse overall top10,data stats keyboardMouse overall top12,data stats keyboardMouse overall top25,data stats keyboardMouse overall kills,data stats keyboardMouse overall killsPerMin,data stats keyboardMouse overall killsPerMatch,data stats keyboardMouse overall deaths,data stats keyboardMouse overall kd,data stats keyboardMouse overall matches,data stats keyboardMouse overall winRate,data stats keyboardMouse overall minutesPlayed,data stats keyboardMouse overall playersOutlived,data stats keyboardMouse overall lastModified,data stats keyboardMouse solo score,data stats keyboardMouse solo scorePerMin,data stats keyboa

In [19]:
#Top Players and Friends
table.to_csv('./Datasets/top_and_friends', index=False)

## Top 100 Highest Overall Earnings

In [20]:
#Web-Scrap: Top 100 Highest Overall Earnings
response= requests.get('https://www.esportsearnings.com/players/highest-overall')
html = response.content
soup = BeautifulSoup(html, "lxml")
table = pd.read_html(html, attrs={'class' : 'detail_list_table'})[0]

In [21]:
# Replacing Header with Top Row - Just run once
new_header = table.iloc[0] 
table = table[1:] 
table.columns = new_header

In [22]:
table.rename(columns={'NaN':'Rank Position'}, inplace=True)

In [23]:
#Top 100 Highest Overall Earnings - Export csv
table.to_csv('./Datasets/highest_earnings_all_games.csv', index=False)

In [24]:
table #Top 100 Highest Overall Earnings

,NaN,Player ID,Player Name,Total (Overall),Highest Paying Game,Total (Game),% of Total
1,1.0,N0tail,Johan Sundstein,"$6,940,222.80",Dota 2,"$6,928,496.58",99.83%
2,2.0,JerAx,Jesse Vainikka,"$6,470,548.78",Dota 2,"$6,470,000.02",99.99%
3,3.0,ana,Anathan Pham,"$6,000,411.96",Dota 2,"$6,000,411.96",100.00%
4,4.0,Ceb,SÃ©bastien Debs,"$5,530,197.41",Dota 2,"$5,530,197.41",100.00%
5,5.0,Topson,Topias Taavitsainen,"$5,448,502.57",Dota 2,"$5,448,502.57",100.00%
...,...,...,...,...,...,...,...
96,96.0,karrigan,Finn Andersen,"$976,822.11",Counter-Strike: Global Offensive,"$945,035.39",96.75%
97,97.0,NAF,Keith Markovic,"$972,165.66",Counter-Strike: Global Offensive,"$972,165.66",100.00%
98,98.0,Duke,"Lee, Ho Seong","$954,620.62",League of Legends,"$954,620.62",100.00%
99,99.0,ELiGE,Jonathan Jablonowski,"$921,346.42",Counter-Strike: Global Offensive,"$920,096.42",99.86%


## Top 100 Highest Overall Earnings - Brazil

In [25]:
#Top 100 Highest Earnings for Brazil
response= requests.get('https://www.esportsearnings.com/countries/br')
html = response.content
soup = BeautifulSoup(html, "lxml")
table2 = pd.read_html(html, attrs={'class' : 'detail_list_table'})[0]
#Turning the first row into header
new_header = table2.iloc[0] 
table2 = table2[1:] 
table2.columns = new_header

In [26]:
table2

,NaN,Player ID,Player Name,Total (Overall),Highest Paying Game,Total (Game),% of Total
1,1.0,FalleN,Gabriel Toledo,"$1,066,974.03",Counter-Strike: Global Offensive,"$1,059,070.54",99.26%
2,2.0,TACO,EpitÃ¡cio de Melo,"$1,062,989.89",Counter-Strike: Global Offensive,"$1,062,989.89",100.00%
3,3.0,fer,Fernando Alvarenga,"$1,054,075.52",Counter-Strike: Global Offensive,"$1,053,170.54",99.91%
4,4.0,coldzera,Marcelo David,"$1,002,401.46",Counter-Strike: Global Offensive,"$1,002,401.46",100.00%
5,5.0,fnx,Lincoln Lau,"$424,984.20",Counter-Strike: Global Offensive,"$386,448.82",90.93%
...,...,...,...,...,...,...,...
96,96.0,Goken,- -,"$40,000.00",CrossFire,"$40,000.00",100.00%
97,97.0,Revolta,Gabriel Henud,"$39,983.40",League of Legends,"$39,983.40",100.00%
98,98.0,iDk,Victor Torraca,"$39,265.43",Counter-Strike: Global Offensive,"$39,265.43",100.00%
99,99.0,technoviking46,Gustavo Trevisan,"$39,200.00",Fortnite,"$39,200.00",100.00%


In [27]:
#Top 100 Highest Earnings for Brazil - Export csv
table2.to_csv('./Datasets/highest_earnings_all_games_brazil.csv', index=False)

## Insights:
    - Compare if the player which wins more money is the player with more kills per minute; 
    - Compare if the player which wins more money is the player with more kills per match;
    - Analysing how many players with earns more money, earn it playing fortnite;
    - Compare wich of my friends kills more, and compare us with pro players;
    - Analyse how many brazilians it's on the top 100 of the world;